In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA
from functions import lasso_loss, logreg_loss
from scipy.linalg import hadamard

In [2]:
n_samples = 1000
n_test = 100
n = 10
p = 8
model = 'logreg'

loss = {'lasso': lasso_loss, 'logreg': logreg_loss}[model]

H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = np.random.randn(p, n_samples)
D = np.random.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = np.random.randn(n, n_samples)

z_test = np.random.randn(p, n_test)
eps_test = np.random.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if model == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = lbda_max * 0.1
print(X.shape)

(10, 1000)


In [11]:
ista = LISTA(D, lbda, 1000, model=model)
print(loss(np.zeros((p, n_samples)), X, D, lbda))
f_star = loss(ista.transform(X), X, D, lbda)
print(f_star)

6.931471805599452
5.338979950110424


In [12]:
n_layers = 3
lista = LISTA(D, lbda, n_layers, model=model)
loss(lista.transform(X), X, D, lbda)

5.509749111526015

In [13]:
lista.fit(X, l_rate=1e-1, max_iter=1000, verbose=True)

it 0, loss = 5.510e+00, grad = 1.33e-01
it 100, loss = 5.433e+00, grad = 5.17e-02
it 200, loss = 5.420e+00, grad = 2.37e-02
it 300, loss = 5.418e+00, grad = 1.43e-02
it 400, loss = 5.417e+00, grad = 1.25e-02
it 500, loss = 5.417e+00, grad = 1.20e-02
it 600, loss = 5.417e+00, grad = 1.17e-02
it 700, loss = 5.417e+00, grad = 1.17e-02
it 800, loss = 5.417e+00, grad = 1.16e-02
it 900, loss = 5.417e+00, grad = 1.16e-02


In [14]:
lista_one = LISTA(D, lbda, 1, model=model)
lista_one.fit(X, l_rate=1e-2, max_iter=1000, batch_size=1000, verbose=True)

it 0, loss = 5.826e+00, grad = 1.78e-01
it 100, loss = 5.796e+00, grad = 1.65e-01
it 200, loss = 5.771e+00, grad = 1.53e-01
it 300, loss = 5.749e+00, grad = 1.43e-01
it 400, loss = 5.730e+00, grad = 1.35e-01
it 500, loss = 5.713e+00, grad = 1.27e-01
it 600, loss = 5.697e+00, grad = 1.20e-01
it 700, loss = 5.684e+00, grad = 1.14e-01
it 800, loss = 5.671e+00, grad = 1.09e-01
it 900, loss = 5.660e+00, grad = 1.04e-01


In [15]:
f_star = loss(LISTA(D, lbda, 1000, model=model).transform(X_test), X_test, D, lbda)
f0 = loss(np.zeros_like(z_test), X_test, D, lbda)

In [16]:
ista = LISTA(D, lbda, n_layers, model=model)
print('Avg test loss using ISTA with %d iterations: %.2e' %
      (n_layers, (loss(ista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using ISTA with 3 iterations: 1.12e-01


In [17]:
print('Avg test loss using LISTA with %d layers: %.2e' %
      (n_layers, (loss(lista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using LISTA with 3 layers: 5.46e-02
